In [1]:
!pip install transformers trl peft bitsandbytes accelerate datasets safetensors -U

In [2]:
from huggingface_hub import notebook_login
from google.colab import userdata
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, LoraConfig, PeftConfig, set_peft_model_state_dict
from safetensors.torch import load_file

In [4]:
print("Unzipping model files...")
!unzip -o /content/tiny-conversational-assistant-final.zip -d /content/

# --- Step 4: Automatically Find the Correct Adapter Path ---
import os
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
print("\nSearching for the adapter_config.json file...")
find_command_output = !find /content -name "adapter_config.json"
if not find_command_output:
    raise FileNotFoundError("ERROR: Could not find 'adapter_config.json' after unzipping.")
correct_config_path = find_command_output[0]
local_adapter_path = os.path.dirname(correct_config_path)
print(f"✅ Found adapter folder at: {local_adapter_path}")

Unzipping model files...
Archive:  /content/tiny-conversational-assistant-final.zip
   creating: /content/content/tiny-conversational-assistant-final/
  inflating: /content/content/tiny-conversational-assistant-final/training_args.bin  
  inflating: /content/content/tiny-conversational-assistant-final/README.md  
  inflating: /content/content/tiny-conversational-assistant-final/adapter_config.json  
  inflating: /content/content/tiny-conversational-assistant-final/adapter_model.safetensors  
  inflating: /content/content/tiny-conversational-assistant-final/special_tokens_map.json  
  inflating: /content/content/tiny-conversational-assistant-final/tokenizer.model  
  inflating: /content/content/tiny-conversational-assistant-final/tokenizer.json  
  inflating: /content/content/tiny-conversational-assistant-final/tokenizer_config.json  
  inflating: /content/content/tiny-conversational-assistant-final/chat_template.jinja  

Searching for the adapter_config.json file...
✅ Found adapter fol

In [6]:
base_model_hub_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
local_adapter_path = "/content/content/tiny-conversational-assistant-final"

print("Loading FINE-TUNED model...")
# Load the base model first
ft_base_model = AutoModelForCausalLM.from_pretrained(
    base_model_hub_id,
    dtype=torch.bfloat16,
    trust_remote_code=True,
)
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_hub_id)
tokenizer.pad_token = tokenizer.eos_token
# Manually load and apply adapters
config_path = f"{local_adapter_path}/adapter_config.json"
with open(config_path, 'r') as f:
    adapter_config_data = json.load(f)
peft_config = LoraConfig(**adapter_config_data)
ft_model = PeftModel(ft_base_model, peft_config)
weights_path = f"{local_adapter_path}/adapter_model.safetensors"
adapter_weights = load_file(weights_path)
set_peft_model_state_dict(ft_model, adapter_weights)
ft_model = ft_model.merge_and_unload()
ft_model.to("cuda")
print("✅ Fine-tuned model loaded successfully!")


# --- Step 6: Load the ORIGINAL Base Model for Comparison ---
print("\nLoading ORIGINAL BASE model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_hub_id,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
print("✅ Base model loaded successfully!")

Loading FINE-TUNED model...
✅ Fine-tuned model loaded successfully!

Loading ORIGINAL BASE model...
✅ Base model loaded successfully!


In [7]:
test_set = [
    {"id": 1, "task": "Ask professor for feedback on a draft", "incoming": "N/A", "request": "an email to Professor Smith asking if he would be available to provide brief feedback on my thesis draft next week."},
    {"id": 2, "task": "Politely decline a social invitation", "incoming": "Hey, we're all going out for drinks on Friday night, you should come!", "request": "a polite text declining the invitation because I have to study for an exam."},
    {"id": 3, "task": "Follow up on a job application", "incoming": "N/A", "request": "a professional follow-up email about the Data Science Intern position I applied for last week."},
    {"id": 4, "task": "Apologize for missing a group meeting", "incoming": "Hey, where were you for the group meeting today? We needed your input.", "request": "an apologetic but concise reply, explaining I had a conflicting appointment and asking when I can sync up."},
    {"id": 5, "task": "Request a letter of recommendation", "incoming": "N/A", "request": "a formal email to Professor Davis, whom I took Advanced Algorithms with, asking for a letter of recommendation for a Master's program."},
    {"id": 6, "task": "Confirm meeting details", "incoming": "Just confirming our project discussion tomorrow.", "request": "a reply asking to confirm the time and the meeting link."},
    {"id": 7, "task": "Handle a group project conflict", "incoming": "I've done my part of the project, but I don't think a final section is good. We should rewrite it.", "request": "a diplomatic response that acknowledges their work but suggests discussing the section as a group before making major changes."},
    {"id": 8, "task": "Ask a clarifying question about an assignment", "incoming": "N/A", "request": "an email to my TA asking for clarification on the required formatting for the final paper in CS101."},
    {"id": 9, "task": "Send a thank-you note after an interview", "incoming": "N/A", "request": "a brief thank-you email to Ms. Chen after an interview for the summer internship role."},
    {"id": 10, "task": "Respond to a scheduling conflict", "incoming": "Can you meet at 4 PM today?", "request": "a reply stating that I can't do 4 PM but I am free anytime before noon."},
]


# --- Step 8: Define the Agent and Run Evaluation ---
def run_inference(model_to_use, user_request, incoming_message):
    prompt = (
        f"### Human:\nI received the following message: \"{incoming_message}\". "
        f"Please help me draft a response that is {user_request}.\n\n"
        f"### Assistant:\n"
    )
    if incoming_message == "N/A": # Handle cases with no incoming message
        prompt = f"### Human:\nPlease help me draft a message that is {user_request}.\n\n### Assistant:\n"

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to("cuda")
    outputs = model_to_use.generate(**inputs, max_new_tokens=256, eos_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_response = response.split("### Assistant:")[-1].strip()
    return cleaned_response.split("### Human:")[0].strip()

# Store results for the report
evaluation_results = []
for item in test_set:
    print(f"\n--- Running Test Case #{item['id']}: {item['task']} ---")

    # Generate with fine-tuned model
    ft_response = run_inference(ft_model, item['request'], item['incoming'])
    print("\n  >> Fine-Tuned Model Response:")
    print(ft_response)

    # Generate with base model
    base_response = run_inference(base_model, item['request'], item['incoming'])
    print("\n  >> Base Model Response:")
    print(base_response)

    evaluation_results.append({
        "ID": item['id'],
        "Task": item['task'],
        "Fine-Tuned Response": ft_response,
        "Base Model Response": base_response
    })


--- Running Test Case #1: Ask professor for feedback on a draft ---

  >> Fine-Tuned Model Response:
Subject: Request for Feedback on Thesis Draft

Dear Professor Smith,

I hope this email finds you well. I am writing to request your feedback on my thesis draft, which I have been working on for the past few weeks

  >> Base Model Response:
Dear Professor Smith,

I hope this email finds you well. I am writing to request your feedback on my thesis draft that I submitted to you last week. I am excited to share my work with you and would love to receive your valuable insights on how to improve it.

As you know, my thesis is about [insert topic], and I have been working on it for the past few months. I have made significant progress, but I would like your guidance to ensure that I have covered all the essential aspects of the topic.

I would appreciate your feedback on the following aspects:

1. Introduction: I would like your opinion on how to make the introduction stand out and grab the 